In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd

import settings as conf
import metadata
from utils import is_number, chunker
from results.multixcan import MXPhenoInfo, MXPhenoResults

# fastENLOC reading functions

In [3]:
with open(conf.FASTENLOC_GTEX_TISSUES_FILE, 'r') as f:
    FASTENLOC_TISSUES_NAMES = set([x.strip() for x in f.readlines()])

In [4]:
# create summary of files
_path = os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], '*.enloc.sig.out.gz')
display(_path)
all_fastenloc_results_files = glob(_path)

'/mnt/phenomexcan_base/results/fastenloc/rapid_gwas_project/*.enloc.sig.out.gz'

In [5]:
PHENO_CODES = set(metadata.RAPID_GWAS_PHENO_INFO.index)

In [12]:
len(PHENO_CODES)

4359

In [13]:
phenos = []
tissues = []

for f in all_fastenloc_results_files:
    split = f.split('-')
    
    tiss = split[-1].split('.')[0]
    if tiss not in FASTENLOC_TISSUES_NAMES:
        tiss = '-'.join(split[-2:]).split('.')[0]
    
    phen = split[1]
    if phen not in PHENO_CODES:
        phen = '-'.join(split[1:3])
    
    phenos.append(phen)
    tissues.append(tiss)

results_summary = pd.DataFrame({'pheno': phenos, 'tissue': tissues, 'file': all_fastenloc_results_files})

In [16]:
display(results_summary.shape)
assert results_summary.shape[0] == int(conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

(198401, 3)

In [17]:
results_summary.head()

,pheno,tissue,file
0,J15,Adrenal_Gland,/mnt/phenomexcan_base/results/fastenloc/rapid_...
1,D03,Pancreas,/mnt/phenomexcan_base/results/fastenloc/rapid_...
2,20003_1141180392,Artery_Coronary,/mnt/phenomexcan_base/results/fastenloc/rapid_...
3,Z01,Muscle_Skeletal,/mnt/phenomexcan_base/results/fastenloc/rapid_...
4,20003_1141146138,Adrenal_Gland,/mnt/phenomexcan_base/results/fastenloc/rapid_...


In [18]:
results_summary['pheno'].unique().shape

(4049,)

In [19]:
assert np.all([t in PHENO_CODES for t in results_summary['pheno'].unique()])

In [20]:
assert results_summary['tissue'].unique().shape[0] == 49

In [22]:
assert np.all([t in FASTENLOC_TISSUES_NAMES for t in results_summary['tissue'].unique()])

In [23]:
_all_tissues_in_results = results_summary['tissue'].unique()
assert np.all([t in _all_tissues_in_results for t in FASTENLOC_TISSUES_NAMES])

In [24]:
assert len(all_fastenloc_results_files) == int(conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

In [26]:
def read_fe(filename): 
    fe_data = pd.read_csv(filename, sep='\s+', usecols=[0, 5], header=None).rename(columns={0: 'gene_cluster', 5: 'rcp'})
    fe_data = fe_data = fe_data.assign(gene=fe_data['gene_cluster'].apply(lambda x: x.split(':')[0])) 
    fe_data = fe_data.assign(gene_cluster=fe_data['gene_cluster'].apply(lambda x: x.split(':')[1]))
    
    fe_max = fe_data.groupby('gene')['rcp'].sum()
    return fe_max

def read_pheno(pheno): 
    _tmp = results_summary[results_summary['pheno'] == pheno] 
    if _tmp.shape[0] == 0: 
        return None 
     
    _res = {} 
    _all_genes = set() 
    for idx, _data in _tmp.iterrows(): 
        tissue_data = read_fe(_data.file) 
        _res[_data.tissue] = tissue_data 
        _all_genes.update(tissue_data.index) 
    
    return pd.DataFrame(_res, index=_all_genes).fillna(-1).max(axis=1)

### Testing

In [31]:
# testing
t = read_fe(os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], 'fastenloc-J15-Whole_Blood.enloc.sig.out.gz'))

In [32]:
t.sort_values(ascending=False).head()

gene
ENSG00000173930    1.869481e-07
ENSG00000260329    5.090216e-08
ENSG00000001561    2.257609e-08
ENSG00000231769    2.001572e-08
ENSG00000175390    1.147284e-08
Name: rcp, dtype: float64

In [33]:
_gid = metadata.GENE_NAME_TO_ID_MAP['SLCO4C1']
assert t[_gid] == (1.869e-07 + 1.578e-11 + 5.201e-12 + 1.409e-13 + 2.694e-11)

In [34]:
_gid = metadata.GENE_NAME_TO_ID_MAP['RP11-34P13.7']
assert t[_gid] == 0.00, t[_gid]

```parallel -j4 'zcat {} | grep ENSG00000049246' ::: fastenloc-1180-*.gz | column -t | sort -k6 -g```

In [35]:
t = read_pheno('J15')

In [36]:
t.sort_values(ascending=False).head()

ENSG00000116985    0.011245
ENSG00000198754    0.009545
ENSG00000279039    0.002292
ENSG00000173930    0.000965
ENSG00000231769    0.000865
dtype: float64

In [37]:
t.loc['ENSG00000116985']

0.01124506203

In [38]:
assert t.loc['ENSG00000116985'] == (1.124e-02 + 5.425e-07 + 1.153e-08 + 2.864e-06 + 1.644e-06)

# ENLOC reading functions

In [39]:
display(conf.FASTENLOC_RESULTS_DIR['GTEX_GWAS'])

'/mnt/phenomexcan_base/results/fastenloc/gtex_gwas'

In [41]:
def read_enloc(enloc_result_path):
    enloc_filename = enloc_result_path.split('/')[-1]
    phenotype = enloc_filename.split('__PM__')[0]
    tissue = enloc_filename.split('__PM__')[1].split('.enloc.rst.gz')[0]
    
    enloc_result = pd.read_csv(enloc_result_path, sep='\s+')
    enloc_result = enloc_result.assign(gene_id=enloc_result['molecular_qtl_trait'].apply(lambda x: x.split('.')[0]))
    
    return enloc_result.groupby('gene_id')['locus_rcp'].max().rename(tissue)

def read_enloc_pheno(phenotype_code):
    all_tissues_data = {}
    all_genes = set()
    
    all_phenotype_files = glob(os.path.join(conf.FASTENLOC_RESULTS_DIR['GTEX_GWAS'], f'{phenotype_code}__PM__*.enloc.rst.gz'))
    
    for f in all_phenotype_files:
        f_data = read_enloc(f)
        all_tissues_data[f] = f_data
        all_genes.update(f_data.index)
    
    return pd.DataFrame(all_tissues_data, index=all_genes).fillna(-1).max(axis=1)

In [42]:
t = read_enloc_pheno('SSGAC_Education_Years_Pooled')

In [43]:
t.head()

ENSG00000127616    0.000
ENSG00000172053    0.011
ENSG00000186684    0.000
ENSG00000159214    0.001
ENSG00000107862    0.101
dtype: float64

In [44]:
# testing
assert t.loc['ENSG00000106113'] == 0.00
assert t.loc['ENSG00000164050'] == 0.006
assert t.loc['ENSG00000081377'] == 0.317

# Get Rapid GWAS phenotypes

In [45]:
all_smultixcan_files = glob(os.path.join(conf.SMULTIXCAN_RESULTS_DIR['RapidGWASProject'], '*.tsv.gz'))
all_smultixcan_phenotypes = [MXPhenoResults(p) for p in all_smultixcan_files]

In [46]:
assert len(all_smultixcan_files) == len(all_smultixcan_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject']

In [47]:
all_smultixcan_files[:5]

['/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_20096_1_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_2345_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_N49_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_100011_raw_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_5221_ccn30.tsv.gz']

# Get GTEx GWAS phenotypes

In [48]:
_path = os.path.join(conf.SMULTIXCAN_RESULTS_DIR['GTEX_GWAS'], '*_ccn30.txt')
display(_path)
all_extra_results_files = glob(_path)
assert len(all_extra_results_files) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

'/mnt/phenomexcan_base/results/smultixcan/gtex_gwas/*_ccn30.txt'

In [49]:
all_extra_results_files[:5]

['/mnt/phenomexcan_base/results/smultixcan/gtex_gwas/CARDIoGRAM_C4D_CAD_ADDITIVE_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt',
 '/mnt/phenomexcan_base/results/smultixcan/gtex_gwas/Astle_et_al_2016_Platelet_count_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt',
 '/mnt/phenomexcan_base/results/smultixcan/gtex_gwas/Astle_et_al_2016_Monocyte_count_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt',
 '/mnt/phenomexcan_base/results/smultixcan/gtex_gwas/Astle_et_al_2016_Neutrophil_count_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt',
 '/mnt/phenomexcan_base/results/smultixcan/gtex_gwas/Astle_et_al_2016_Lymphocyte_counts_smultixcan_imputed_gwas_gtexv8mashr_ccn30.txt']

In [50]:
_file_pattern = '(?P<code>[^/]+)_smultixcan_imputed_gwas_gtexv8mashr_ccn30\.txt'
all_extra_phenotypes = [MXPhenoResults(p, _file_pattern) for p in all_extra_results_files]
all_extra_phenotypes_plain_names = pd.Index([p.pheno_info.get_plain_name() for p in all_extra_phenotypes])

display(len(all_extra_phenotypes))
assert len(all_extra_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

42

In [51]:
all_extra_phenotypes[0].pheno_info.get_plain_name()

'CARDIoGRAM_C4D_CAD_ADDITIVE'

In [54]:
all_extra_phenotypes_codes = [os.path.basename(f).split('_smultixcan_imputed_gwas_gtexv8mashr_ccn30')[0] for f in all_extra_results_files]
assert len(all_extra_phenotypes_codes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

In [55]:
all_extra_phenotypes_codes[:5]

['CARDIoGRAM_C4D_CAD_ADDITIVE',
 'Astle_et_al_2016_Platelet_count',
 'Astle_et_al_2016_Monocyte_count',
 'Astle_et_al_2016_Neutrophil_count',
 'Astle_et_al_2016_Lymphocyte_counts']

In [56]:
all_extra_phenotypes_codes = set(all_extra_phenotypes_codes)

# Read all results

In [57]:
def read_generic(pheno_code):
    if pheno_code in all_extra_phenotypes_codes:
        return read_enloc_pheno(pheno_code)
    else:
        return read_pheno(pheno_code)

def _get_combined_results(phenos):
    return {pheno.pheno_info.get_plain_name() : read_generic(pheno.pheno_info.pheno_code) for pheno in phenos}

In [58]:
# testing
_tmp = _get_combined_results(all_smultixcan_phenotypes[:4])
assert len(_tmp) == 4

In [59]:
# testing
_tmp = _get_combined_results(all_extra_phenotypes[:4])
assert len(_tmp) == 4

In [60]:
_pending = read_generic('C_MYELOID-LEUKAEMIA')

In [61]:
_pending.head()

ENSG00000244754    1.248390e-10
ENSG00000168092    9.029830e-11
ENSG00000172053    6.092500e-12
ENSG00000232878    3.847000e-12
ENSG00000117215    2.153970e-11
dtype: float64

In [62]:
_pending = read_generic('SSGAC_Education_Years_Pooled')

In [63]:
_pending.head()

ENSG00000127616    0.000
ENSG00000172053    0.011
ENSG00000186684    0.000
ENSG00000159214    0.001
ENSG00000107862    0.101
dtype: float64

In [64]:
def _run_all(phenotype_chunks, n_jobs=20):
    all_results = {}
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        tasks = [executor.submit(_get_combined_results, chunk) for chunk in phenotype_chunks]
        for future in as_completed(tasks):
            res = future.result()
            all_results.update(res)

    return all_results

In [65]:
# phenotype_chunks = chunker(all_smultixcan_phenotypes[:5] + all_extra_phenotypes[:5], 2)
phenotype_chunks = chunker(all_smultixcan_phenotypes + all_extra_phenotypes, 200)

In [66]:
all_results = _run_all(phenotype_chunks)

## Save as DataFrame

In [67]:
assert len(all_results) == conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] + conf.FASTENLOC_EXPECTED_PHENOTYPES['GTEX_GWAS']

In [68]:
fastenloc_genes_associations = pd.DataFrame(all_results)
fastenloc_genes_associations.index.rename('gene_name', inplace=True)

assert fastenloc_genes_associations.index.is_unique

display(fastenloc_genes_associations.shape)
display(fastenloc_genes_associations.head())

(37967, 4091)

,20485-Ever_contemplated_selfharm,6160_2-Leisuresocial_activities_Pub_or_social_club,6139_3-Gas_or_solidfuel_cookingheating_An_open_solid_fuel_fire_that_you_use_regularly_in_winter_time,L12_FOLLICULARNAS-Other_and_unspecified_follicular_disorders,2654_8-Nonbutter_spread_type_details_Other_low_or_reduced_fat_spread,22617_8139-Job_SOC_coding_Assemblers_and_routine_operatives_nec,I9_CARDMPRI-Cardiomyopathies_Primaryintrinsic,20003_1141188442-Treatmentmedication_code_glucosamine_product,RHEUMA_SEROPOS_OTH-Otherunspecified_seropositiverheumatoid_arthritis,22601_11523251-Job_coding_office_manager_accountpayrolls_manager_administration_manager_litigationsolicitors_office_manager,...,PGC_ADHD_EUR_2017,BCAC_ER_negative_BreastCancer_EUR,SSGAC_Education_Years_Pooled,pgc.scz2,MAGNETIC_HDL.C,MAGIC_ln_FastingInsulin,Astle_et_al_2016_Sum_eosinophil_basophil_counts,SSGAC_Depressive_Symptoms,Jones_et_al_2016_SleepDuration,Astle_et_al_2016_Red_blood_cell_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.000008,0.000505,0.000305,3.300140e-11,0.000039,3.736700e-07,0.000010,0.000017,0.000044,2.399400e-07,...,0.0,NaN,0.001,0.000,NaN,NaN,0.001,NaN,NaN,0.001
ENSG00000000457,0.000131,0.002876,0.003605,8.840290e-11,0.000203,7.714700e-07,0.000195,0.000601,0.000245,5.047600e-06,...,NaN,NaN,0.000,NaN,NaN,NaN,0.000,NaN,NaN,0.000
ENSG00000000460,0.000014,0.000975,0.005453,9.497296e-11,0.000147,1.070900e-06,0.000029,0.000224,0.004665,3.258560e-06,...,NaN,NaN,0.000,NaN,NaN,NaN,0.000,NaN,NaN,0.000
ENSG00000000938,0.000019,0.000449,0.014776,8.855560e-11,0.000091,6.395000e-07,0.000090,0.000767,0.000453,5.325200e-07,...,NaN,NaN,NaN,0.002,NaN,NaN,0.001,NaN,NaN,NaN
ENSG00000000971,0.000006,0.000674,0.001918,4.448820e-10,0.000057,1.796000e-07,0.000044,0.000049,0.000212,9.644000e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,0.000


In [69]:
# check genes id format
_tmp = pd.Series(fastenloc_genes_associations.index).apply(len).value_counts()
display(_tmp)
assert _tmp.shape[0] == 1
assert _tmp.index[0] == 15

15    37967
Name: gene_name, dtype: int64

In [70]:
# check columns data type
_tmp = fastenloc_genes_associations.dtypes.value_counts()
display(_tmp)
assert _tmp.shape[0] == 1

float64    4091
dtype: int64

In [71]:
fastenloc_genes_associations.isna().sum().sum()

3341082

In [72]:
fastenloc_genes_associations.dropna(axis=0, how='all').shape

(37967, 4091)

In [74]:
display(fastenloc_genes_associations.shape)

(37967, 4091)

In [76]:
assert fastenloc_genes_associations.shape == (37967, conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] + conf.FASTENLOC_EXPECTED_PHENOTYPES['GTEX_GWAS'])

In [77]:
# some testing
t = fastenloc_genes_associations['SSGAC_Education_Years_Pooled']

In [78]:
t.head()

gene_name
ENSG00000000419    0.001
ENSG00000000457    0.000
ENSG00000000460    0.000
ENSG00000000938      NaN
ENSG00000000971      NaN
Name: SSGAC_Education_Years_Pooled, dtype: float64

In [79]:
assert t.loc['ENSG00000106113'] == 0.00
assert t.loc['ENSG00000164050'] == 0.006
assert t.loc['ENSG00000081377'] == 0.317
assert pd.isnull(t.loc['ENSG00000000938'])

In [80]:
# Save
fastenloc_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, f'fastenloc-torus-rcp.pkl.xz')
display(fastenloc_genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/fastenloc-torus-rcp.pkl.xz'

In [81]:
fastenloc_genes_associations.to_pickle(fastenloc_genes_associations_filename)